In [1]:
# Import libraries
import numpy as np
import pandas as pd
import psycopg2
import getpass

In [2]:
# Load the database connection details
_data = np.load('data/db_details.npy', allow_pickle=True).tolist()
db_details = _data['db_details']
user = db_details['user']
host = db_details['host']
port = db_details['port']
dbname = db_details['dbname']
schema = db_details['schema']
    
# Connect to the database
conn = psycopg2.connect(dbname=dbname, user=user, host=host, port=port, password=getpass.getpass(prompt='Password:'.format(user)))
cur = conn.cursor()
cur.execute('SET search_path TO {}'.format(schema))
conn.commit()

Password: ··········


In [3]:
# Select data for all ICU admissions
query = \
"""
SELECT
    i.subject_id,
    i.hadm_id,
    i.icustay_id,
    p.gender,
    p.dob,
    ROUND((CAST(i.intime AS date) - CAST(p.dob AS date))/365.242,2) AS age,
    i.intime,
    i.outtime,
    i.los,
    CASE
        WHEN p.dod_hosp IS NULL THEN 0
        ELSE 1
    END hosp_expire_flag
FROM icustays i INNER JOIN patients p
ON i.subject_id = p.subject_id
"""

# Execute the query and fetch the result
cur = conn.cursor()
cur.execute(query)
conn.commit()
data = cur.fetchall()

# Store the result of the query as a pandas dataframe
labels = ['subject_id', 'hadm_id', 'icustay_id', 'gender', 'dob', 'age', 'intime', 'outtime', 'los', 'hosp_expire_flag']
icu = pd.DataFrame(data, columns=labels)

In [4]:
icu.head()

,subject_id,hadm_id,icustay_id,gender,dob,age,intime,outtime,los,hosp_expire_flag
0,268,110404,280836,F,2132-02-21,65.98,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490,1
1,269,106296,206613,M,2130-09-30,40.10,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788,0
2,270,188028,220345,M,2048-05-26,80.08,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939,0
3,271,173727,249196,F,2074-11-30,45.68,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600,0
4,272,164716,210407,M,2119-11-21,67.10,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202,0


In [5]:
print('subject_id: {}'.format(len(np.unique(icu['subject_id']))))
print('hadm_id: {}'.format(len(np.unique(icu['hadm_id']))))
print('icustay_id: {}'.format(len(np.unique(icu['icustay_id']))))

subject_id: 46476
hadm_id: 57786
icustay_id: 61532
